# Initialisation

In [492]:
import matplotlib
matplotlib.use('Qt5Agg')  # Utiliser le backend Qt5Agg pour Windows
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cv2
import numpy as np
import os
from PIL import Image
import lmfit
#import tifffile
#%matplotlib tk #pour Linux

In [493]:
import scipy.ndimage as ndimage
from sklearn.cluster import DBSCAN

# Importer fichier

In [494]:
path_to_tiff = os.path.join("..", "acquisition", "video_output_carac_150ms_1im_1um.tiff")

tiff = Image.open(path_to_tiff)

# Nombre de frames pas vide

In [495]:
with Image.open(path_to_tiff) as img:
    frame_number = 0
    actual_frames = 0
    try:
        while True:
            frame_number += 1
            if np.sum(np.array(img)) != 0:
                actual_frames += 1
                
            img.seek(frame_number)
    except EOFError:
        print("All frames processed.")

actual_frames

All frames processed.


71

# 1re frame

**Le nombre de frames ignorés n'est pas pris en compte**

In [496]:
frame_index = 0
first_frame = 0
tiff.seek(frame_index)
original_image = np.array(tiff)

while np.sum(original_image) == 0:
    frame_index += 1
    tiff.seek(frame_index)
    original_image = np.array(tiff)
    first_frame += 1
    print(frame_index)




1


# Traitement d'image

In [497]:
clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(30, 30))
preprocessed = clahe.apply(original_image)

blurred = cv2.medianBlur(preprocessed, 115)
preprocessed2 = cv2.subtract(preprocessed, blurred)

# Apply Non-Local Means Denoising
img = cv2.fastNlMeansDenoising(preprocessed2, None, 15, 7, 41)
print(img)

[[ 9 10 10 ... 11 11 11]
 [10 10 10 ... 11 11 11]
 [ 9  9 10 ... 11 12 12]
 ...
 [ 9  9 10 ...  8  8  8]
 [ 8  9  9 ...  8  8  8]
 [ 8  9  9 ...  8  9  8]]


# Sélection du point à tracker

In [498]:
def crop(img, x, y, crop_size_x, crop_size_y):
    x_start = int(x - crop_size_x // 2)
    x_end = int(x + crop_size_x // 2)
    y_start = int(y - crop_size_y // 2)
    y_end = int(y + crop_size_y // 2)

    return img[y_start:y_end, x_start:x_end]

In [499]:
# Display the image and let the user select a point interactively
fig, ax = plt.subplots()
ax.imshow(img, origin='lower', cmap='gray')  # Use 'gray' for better visibility of grayscale images
plt.title(f"Frame {frame_index}: Select a point")

# Ask for a point to be selected
print("Please click on the point you want to select.")
x, y = plt.ginput(1)[0]  # This will get the coordinates of the clicked point
print(f"Selected point: ({x}, {y})")
plt.close()

crop_sze = crop_sze_x = crop_sze_y = 50

Please click on the point you want to select.
Selected point: (865.6038961038963, 654.6298701298701)


Debogueur

In [500]:
def visionneur(frame, titre):
    plt.figure(figsize=(10, 5))
    plt.clf() 
    plt.imshow(frame, origin='lower', cmap='gray')
    plt.title(titre)
    plt.colorbar()  
    plt.show()

# Fit gaussien sur le point sélectionné

In [501]:
def prepare_data(x, y, z):
    return (x.flatten(), y.flatten()), z.flatten()

In [502]:
def gaussian_2d(xy, amplitude, x0, y0, sigma_x, sigma_y, offset):
    x, y = xy
    a = 1 / (2 * sigma_x**2)
    b = 1 / (2 * sigma_y**2)
    return offset + amplitude * np.exp(- (a * (x - x0)**2 + b * (y - y0)**2))

In [503]:
def localisateur_gaussien(intensity_grid, centre):
    x = np.arange(intensity_grid.shape[0])
    y = np.arange(intensity_grid.shape[1])
    X, Y = np.meshgrid(x, y)

    # Préparer les données pour le fit
    (xdata, ydata), zdata = prepare_data(X, Y, intensity_grid)
    model = lmfit.Model(gaussian_2d)
    max_idx = np.unravel_index(np.argmax(intensity_grid), intensity_grid.shape)
    initial_x0 = x[max_idx[0]]
    initial_y0 = y[max_idx[1]]

    # Définir les paramètres du modèle
    params = model.make_params(
        amplitude=np.max(intensity_grid),
        x0=initial_x0,
        y0=initial_y0,
        sigma_x=1,
        sigma_y=1,
        offset=2
    )

    # Effectuer l'ajustement
    result = model.fit(zdata, params, xy=(xdata, ydata))
    
    x_position = result.params['x0'].value + centre[0] - crop_sze_x/2
    y_position = result.params['y0'].value + centre[1] - crop_sze_y/2

    return [x_position, y_position], result.params['sigma_x'].value, result.params['sigma_y'].value

# Process d'image (enlever le bruit)

**semble faire du trouble**

In [504]:
def denoise(image):
    clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(30, 30))
    preprocessed = clahe.apply(image)
    
    blurred = cv2.medianBlur(preprocessed, 115)
    preprocessed2 = cv2.subtract(preprocessed, blurred)
    
    return cv2.fastNlMeansDenoising(preprocessed2, None, 15, 7, 41)

# Identifier la particule et donner sa nouvelle position

In [505]:
def empreinte_digitale(image, threshold=12, eps=3, min_samples=1):
    # 1. Observer chaque particule approximativement
    #neighborhood_size = 3  
    #local_max = ndimage.maximum_filter(image, size=neighborhood_size)
    visionneur(image, f'particule choisie')

    maxima = (image > threshold)
    
    coordinates = np.column_stack(np.where(maxima))  
    if coordinates.shape[0] == 0:
        print("Aucune particule détectée.")
        return [], image

    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(coordinates)
    unique_labels = np.unique(labels)
    size=0

    # 2. Déterminer l'empreinte digitale de notre particule
    for label in unique_labels:
        if label != -1:  # -1 correspond au bruit dans DBSCAN
            cluster_points = coordinates[labels == label]
            plt.scatter(cluster_points[:, 1], cluster_points[:, 0], label=f'Particule {label}')

            if cluster_points.shape[0]>size:
                size = cluster_points.shape[0]
                luminosity = np.mean(image[cluster_points[:, 0], cluster_points[:, 1]])

    seuil=np.max(image)*0.4
    print(seuil)
    # 3. Afficher les résultats et filtrer les particules selon leur taille et luminosité
    plt.imshow(image, origin='lower', cmap='gray')
    plt.title(f"Empreinte digitale de la particule, lum={luminosity} et taille={size}")
    plt.legend()
    plt.colorbar()  
    plt.show()

    return size, luminosity, seuil

In [506]:
def detect_nbre_particles(image, threshold, eps=3, min_samples=1):
    # 1. Appliquer un filtre de voisinage pour détecter les maxima locaux
    neighborhood_size = 3  # Taille du voisinage pour détecter les maxima locaux
    local_max = ndimage.maximum_filter(image, size=neighborhood_size)
    # 2. Comparer l'image originale et les maxima locaux pour identifier les vrais maxima
    maxima = (image == local_max) & (image > threshold)

    # 3. Extraire les coordonnées des maxima (particules)
    coordinates = np.column_stack(np.where(maxima))  # Extraire les indices des pixels maximaux
    if coordinates.shape[0] == 0:
        print("Aucune particule détectée.")
        visionneur(image, f'détection du nombre de particules')

        return [], image

    # 4. Appliquer DBSCAN pour regrouper les particules proches (cluster les maxima détectés)
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(coordinates)

    #5. Déterminer le nombre de particules
    unique_labels = np.unique(labels)
    nb_particules = len(unique_labels)
        
    # 6. Afficher les résultats et filtrer les particules selon leur taille et luminosité
    plt.imshow(image, origin='lower', cmap='gray')
    plt.title("Détection des particules")
    
    positions=[]
    for label in unique_labels:
        if label != -1:  # -1 correspond au bruit dans DBSCAN
            cluster_points = coordinates[labels == label]
            plt.scatter(cluster_points[:, 1], cluster_points[:, 0], label=f'Particule {label}')
#        if nb_particules > 1:
        positions.append((np.mean(cluster_points[:, 1]), np.mean(cluster_points[:, 0])))
        

    plt.colorbar()  
    plt.legend()
    plt.show()

    # Retourner les coordonnées des particules filtrées
    return nb_particules, positions

In [507]:
def identificateur_particles(image, ref_size, ref_luminosity, threshold=25, eps=3, min_samples=1):
    # 1. Obtenir le plus d'info sur la particule
    maxima = (image > threshold)
    
    coordinates = np.column_stack(np.where(maxima))  
    if coordinates.shape[0] == 0:
        print("Mauvais re-crop empêche identification de la particule.")
        return [], image

    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(coordinates)
    unique_labels = np.unique(labels)

    # 2. Parcourir chaque particule et identifier laquelle est celle qu'on suit
    distances = []
    position_particules = []

    for label in unique_labels:
        if label != -1:  # -1 correspond au bruit dans DBSCAN
            cluster_points = coordinates[labels == label]
            
            size = cluster_points.shape[0]
            luminosity = np.mean(image[cluster_points[:, 0], cluster_points[:, 1]])
            size_distance = abs(size - ref_size)  
            luminosity_distance = abs(luminosity - ref_luminosity)  
            combined_distance = size_distance + luminosity_distance
            distances.append(combined_distance)
            position_particules.append((np.mean(cluster_points[:, 1]), np.mean(cluster_points[:, 0])))

    ecart = np.min(distances)
    position_best_particle = position_particules[np.argmin(distances)]  
    print(f'ecrat: {ecart}')
    # 3. Afficher les résultats et filtrer les particules selon leur taille et luminosité
    plt.imshow(image, origin='lower', cmap='gray')
    plt.title("Détection des particules")
    plt.scatter(position_best_particle[0], position_best_particle[1], color='r', label=f'Particule sélectionnée')
    plt.legend()
    plt.colorbar()  
    plt.show()

    return position_best_particle, ecart

# Faire le crop et fit

In [508]:
def particle_tracker_simple(image, x, y):
    image = denoise(image)

    cropped_img = crop(image, x, y, crop_sze_x, crop_sze_y)
    
    #Gérer plus qu'une particule
    cropped_img = np.array(cropped_img)
    max_index = np.argmax(cropped_img)
    max_coords = np.unravel_index(max_index, cropped_img.shape)

    nouveau_x = x - crop_sze_x // 2 + max_coords[1]
    nouveau_y = y - crop_sze_y // 2 + max_coords[0]
    
    second_crop = crop(image, nouveau_x, nouveau_y, crop_sze_x, crop_sze_y)    # Re-crop autour d'une seule particule

    result_fit = localisateur_gaussien(second_crop, [nouveau_x, nouveau_y])

    x_new, y_new = result_fit[0][0], result_fit[0][1]

    return [result_fit, cropped_img, (x_new, y_new), (result_fit[1], result_fit[2])]

In [509]:
#Placés ici pour pouvoir y faire référence dans particule_tracker
position_list=[]
sigma_list = []
crop_frames = []
big_frames = []

In [510]:
def particle_tracker(image, x, y, taille_initiale, luminosité_initiale, seuil):
    # 1. Ce qu'on voit près de l'ancienne position
    image = denoise(image)

    cropped_img = crop(image, x, y, crop_sze_x, crop_sze_y)
    
    # 2. Déterminer combien il y a de particules et laquelle est la notre
    # Utiliser une reconnaissance de particules et si plus qu'une, alors clic pour choisir
    nb_part, coordonnées = detect_nbre_particles(cropped_img, seuil)
    print(f'coordonnées des {nb_part} particules:{coordonnées}')

    if (nb_part == 0) or (nb_part == []):
        # Display the image and let the user select a point interactively
        fig, ax = plt.subplots()
        ax.imshow(denoise(img), origin='lower', cmap='gray') 
        plt.title(f"Frame {len(position_list)-1}: Select a point")

        # Ajouter une croix rouge à la position donnée
        cx, cy = position_list[len(position_list)-1]
        ax.plot(cx, cy, 'rx', markersize=10) 
        
        # Afficher l'image et demander à l'utilisateur de cliquer
        print("Please click on the point you want to select.")
        nouveau_x, nouveau_y = plt.ginput(1)[0]  # Attente du clic de l'utilisateur (1 point)
        print(f"Selected point: ({x}, {y})")
        
        # Fermer l'affichage après sélection
        plt.close()
    elif nb_part > 1:
        imperfections, emplacements = [] , []
        for num_particule in range(nb_part):
            x_identifier=x - crop_sze_x // 2 + coordonnées[num_particule][0]
            y_identifier=y - crop_sze_y // 2 + coordonnées[num_particule][1]
            crop_pour_identifier = crop(image, x_identifier, y_identifier, crop_sze_x//2, crop_sze_y//2)
            emplacement, ecart = identificateur_particles(crop_pour_identifier, taille_initiale, luminosité_initiale, seuil)
            imperfections.append(ecart)
            emplacements.append(emplacement)
        qualité = np.min(imperfections)
        print(f'écart:{qualité}')
        max_coords = emplacements[np.argmin(imperfections)]  
        nouveau_x = x_identifier - crop_sze_x // 4 + max_coords[0]
        nouveau_y = y_identifier - crop_sze_y // 4 + max_coords[1]
    else:
        cropped_img = np.array(cropped_img)
        max_index = np.argmax(cropped_img)
        max_coords = np.unravel_index(max_index, cropped_img.shape)
    #print(F'coordonnées du max dans le premier crop: ({max_coords[1]},{max_coords[0]})')

        nouveau_x = x - crop_sze_x // 2 + max_coords[1] #Oui c'est inversé à cause du unravel juste avant
        nouveau_y = y - crop_sze_y // 2 + max_coords[0]

    #print(f'(x,y) central du second crop: {nouveau_x, nouveau_y}')
    
    # 3. Crop atour de notre particule et fit dessus
    second_crop = crop(image, nouveau_x, nouveau_y, crop_sze_x//2, crop_sze_y//2)    # Re-crop autour d'une seule particule


    result_fit = localisateur_gaussien(second_crop, [nouveau_x, nouveau_y])
    #print(f'écart-type: ({result_fit[1]}, {result_fit[2]})')

    x_new, y_new = result_fit[0][0], result_fit[0][1]

    x_range = np.arange(crop_sze_x) - crop_sze_x//2 
    y_range = np.arange(crop_sze_y) - crop_sze_y//2
    x_position = x_range + x_new  
    y_position = y_range + y_new 
    plt.imshow(second_crop, origin='lower', cmap='gray', extent=[x_position.min(), x_position.max(), y_position.min(), y_position.max()])
    plt.title("positionnement de la particule")
    plt.scatter(x_new, y_new, color='r', label=f'Position estimée')
    plt.legend()
    plt.colorbar()  
    plt.show()

    return [result_fit, second_crop, (x_new, y_new), (result_fit[1], result_fit[2])]

# Main loop

In [511]:
taille_initiale, luminosité_initiale, seuil = empreinte_digitale(crop(img,x,y,35,35))


#for i in range(first_frame,actual_frames):
for i in range(first_frame,15):
    tiff.seek(i)
    if np.sum(np.array(tiff))==0:
        position_list.append(np.array([np.nan,np.nan]))
        sigma_list.append(np.array([np.nan,np.nan]))
        print("hello")
    else:
        data = particle_tracker(img, x, y, taille_initiale, luminosité_initiale, seuil)
        position_list.append(data[2])
        x,y = position_list[i-first_frame]
        sigma_list.append(data[3])
        tiff.seek(i)
        img = np.array(tiff)
        big_frames.append(img)
        crop_frames.append(data[1])
    print(position_list)


18.8
coordonnées des 1 particules:[(24.5, 25.5)]
[(852.7143890003176, 641.296118842698)]
coordonnées des 2 particules:[(41.0, 29.0), (37.5, 37.5)]
ecrat: 163.7204678362573
ecrat: 119.07201125175808
écart:119.07201125175808
[(852.7143890003176, 641.296118842698), (852.5617121324624, 641.223392712375)]
coordonnées des 1 particules:[(38.0, 38.5)]
[(852.7143890003176, 641.296118842698), (852.5617121324624, 641.223392712375), (852.8350033051489, 641.5175747526624)]
coordonnées des 1 particules:[(33.75, 39.0)]
[(852.7143890003176, 641.296118842698), (852.5617121324624, 641.223392712375), (852.8350033051489, 641.5175747526624), (848.5476272955149, 642.5248135754223)]
coordonnées des 1 particules:[(38.0, 43.2)]
[(852.7143890003176, 641.296118842698), (852.5617121324624, 641.223392712375), (852.8350033051489, 641.5175747526624), (848.5476272955149, 642.5248135754223), (848.6277615183529, 648.0499535263502)]
coordonnées des 2 particules:[(37.0, 31.0), (35.5, 40.5)]
ecrat: 183.38055555555556
ecra

In [512]:
def calculate_msd_with_uncertainty(positions, delta_x, delta_y):
    """
    Calcule les MSD avec propagation des incertitudes analytiques.
    """
    msd = []
    uncertainties = []
    for d in range(1, len(positions)):
        diff_pairs = positions[d:] - positions[:-d]
        dx = delta_x[d:] + delta_x[:-d]
        dy = delta_y[d:] + delta_y[:-d]
        term_x = 2 * (diff_pairs[:, 0]**2) * (dx**2)
        term_y = 2 * (diff_pairs[:, 1]**2) * (dy**2)

        distances_squared = np.sum(diff_pairs**2, axis=1)
        msd.append(np.mean(distances_squared))  
        
        # Propagation des incertitudes
        total_uncertainty = np.mean(term_x + term_y)
        uncertainties.append(np.sqrt(total_uncertainty))
    
    return np.array(msd), np.array(uncertainties)

# Résultats

In [513]:
sigma_list

[(4.81613527641922, 5.615947856004609),
 (3.6169646881911692, 4.754793338362959),
 (3.4416227408166695, 4.442691105424915),
 (3.4410934319648736, 4.237715599059012),
 (3.3515526157746285, 4.571869322139503),
 (2.994411246990326, 4.837143705043078),
 (2.8277029269095557, 5.559860649484158),
 (3.037838117357258, 4.631740412819397),
 (2.9910793070918116, 4.714120074640651),
 (2.8003834951861513, 4.633636832793034),
 (2.843764986049514, 4.632428224865064),
 (2.6932138843516085, 4.848733337004061),
 (2.7876282675043376, 4.544093112556272),
 (2.736707027528531, 3.638045534189175)]

In [514]:
x_plt, y_plt = zip(*position_list)
# Create the plot
plt.figure(figsize=(10, 8))  # Optional: Adjust the figure size
plt.plot(x_plt, y_plt, marker='o', linestyle='-', color='b', label='Connected Points')

# Inverser l'axe Y
plt.gca().invert_yaxis()

# Set grid limits to match the 1440x1080 grid
plt.xlim(0, 1440)
plt.ylim(0, 1080)

# Add labels and title
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('2D Connected Points Plot')
plt.legend()

# Optional: Add grid lines
plt.grid(True)

# Afficher le graphique
plt.show()

## anim toutes les particules

In [515]:
fig, ax = plt.subplots()
img = ax.imshow(big_frames[0], origin='lower', cmap='gray', animated=True)


# Update function
def update(frame):
    img.set_array(frame)
    return img,
    
ani = animation.FuncAnimation(fig, update, frames=big_frames, interval=50, blit=True)
plt.show()

## anim crop

In [516]:
fig, ax = plt.subplots()
img = ax.imshow(crop_frames[0], origin='lower', cmap='gray', animated=True)


def update(frame):
    img.set_array(frame)
    return img,
    
ani = animation.FuncAnimation(fig, update, frames=crop_frames, interval=50, blit=True)
plt.show()